# Convolutional Neural Network - Regression + KFold
### Import Preliminaries

In [1]:
import numpy as np
from keras import models
from keras import layers
from keras.datasets import boston_housing

Using TensorFlow backend.


### Preprocessing the Data

In [2]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

# Normalize the data
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= mean

### Model Generator Function

In [3]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

### Build, Compile, and Train Model within K-Fold Cross Validation

In [4]:
k = 4

num_val_samples = len(train_data)//k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i + 1) * num_val_samples:]], 
        axis =0)
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i + 1) * num_val_samples:]], 
        axis =0)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
             epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, 
                                      verbose=0)
    all_scores.append(val_mae)
    

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


### View Results

In [9]:
all_scores

[2.1407350431574454,
 2.3300966319471303,
 2.9427623914019896,
 2.3556685866695819]

In [10]:
# Mean Score
print('Mean Score',np.mean(all_scores))

Mean Score 2.44231566329
